In [ ]:
#Charger le repertoire des scripts
import sys
import os
sys.path.append(os.path.abspath('./Library'))
import numpy as np
from matplotlib.pyplot import close, show
import matplotlib.pyplot as plt
import xarray as xr
from reef_models import reef_platform, reef, reef_eros
from tools import nlines, readfile
from tools_models import CheckSimu, ZarrName, shore
from Profile import Profile
from Dicts import Dicos
from datetime import datetime as dtime
from os import path
import xsimlab as xs
from Dict_models import DicoModels
from scipy.interpolate import interp1d
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

## Model setup

You only have to run the following cell, do not modify

In [ ]:
dico = Dicos()
class REEF:
    
    def __init__(self):
        # Define the fixed parameters
        self.input_vars = {
            # Water height for open ocean
            'hfactor__how': 2,
            # uniform spacing
            'grid__spacing': 1,
            # Grid width factor. Should be similar to hmax 
            'grid__dmax': 20,
            # Coefficient for erosion efficiency, sea-bed
            'eros__beta1': 0.1,
            # Coefficient for erosion efficiency, cliff retreat
            'eros__beta2': 1,
            # Height of notch for volume eroded during cliff retreat
            'eros__hnotch': 1,
            # Repos angle
            'depot__repos': 15e-2,
        }

    def eros_params(self, SL, U, slop, Ev, hwb=3.):
        self.input_vars.pop('hfactor__how', None)
        self.input_vars.pop('depot__repos', None)
        self.input_vars.update({
            ## Usefull parameters to test. Everything in meters and years.
            # filename for RSL reconstruction
            'SLstory__RSLin': SL,
            # vertical land motion rate. [-10 - 2]e-3 (huge values, you can test before with 0.something e-3)
            'vertical__u': U,
            # initial slope of the substrate. [2 - 8]
            'grid__slopi': slop,          
            # Eroded volume. [50 - 1000]e-3
            'eros__Ev': Ev,
            # Water height for wave base (the waves will start to erode at this depth). [1 - 10]
            'eros__hwb': float(hwb),
        })
    
    def reef_params(self, SL, U, slop, Gm, Ev, hmax=20, Dbar=100, hwb=3.):
        self.__init__()
        # Complete the dictionnary of fixed parameters with the ones to test
        self.input_vars.update({
            ## Usefull parameters to test. Everything in meters and years.
            # filename for RSL reconstruction
            'SLstory__RSLin': SL,
            # vertical land motion rate. [-10 - 2]e-3 (huge values, you can test before with 0.something e-3)
            'vertical__u': U,
            # initial slope of the substrate. [2 - 8]
            'grid__slopi': slop,
            # maximum reef growth rate. [2 - 15]
            'construct__Gm': Gm,            
            # Eroded volume. [50 - 1000]e-3
            'eros__Ev': Ev,
            # maximum water height for reef growth. [10 - 50] 
            'construct__hmax': hmax,
            # width of wave action (controls the width of the coral reef). [10 - 3000] 
            'hfactor__Dbar': Dbar,
            # Water height for wave base (the waves will start to erode at this depth). [1 - 10]
            'eros__hwb': float(hwb),
        })
        
    def run(self, dt, save, construction=True):
        self.save = save
        # Define the output of the model
        if save==True:
            output = {
                'init__x'       : None,
                'profile__z'    : 'time',
                'sealevel__asl' : 'time',
                'profile__xmin' : None,
                'profile__xmax' : None,
                'depot__dS'     : 'time',
                'construct__dG' : 'time',
            }
        elif save==False:
            output={
                'init__x'       : None,
                'profile__z'    : None,
                'sealevel__asl' : None,
                'profile__xmin' : None,
                'profile__xmax' : None,
            }
        else:
            raise Exception("Wrong format for 'save' variable. Should be True or False")

        if construction==True:
            model_type = reef
            model_name = 'reef'
        elif construction==False:
            model_type = reef_eros
            model_name = 'reef_eros'
        else:
            raise Exception("Wrong format for construction variable. Should be True or False")
            
        tmax = nlines(dico.path_SLcurves+self.input_vars['SLstory__RSLin'])*1e3
        print("Simu of", tmax/1000, "ky")
        # Create xsimlab setup
        ds_in = xs.create_setup(
            model=model_type,
            clocks={'time' : np.arange(0., tmax+dt, dt)},
            master_clock = 'time',
            input_vars = self.input_vars,
            output_vars = output
        )
        ds_in.attrs['model_name'] = model_name
        ds_in.attrs['store'] = 'Profile'

        # Run the model
        t0=dtime.today()
        dm   = DicoModels()
        with dm.models[ds_in.model_name]:
            self.ds_out = (ds_in   
              .xsimlab.run()
                  )
        print('Duration', dtime.today()-t0)
        return self.ds_out

    def plot(self, compare=False):
        if self.save==True:
            print('Starting plot')
            fig = Profile(self.ds_out)
            fig.ProfileName('.png')
            fig.PlotProfile()
            fig.PlotASL()
            fig.WriteParams()
            fig.Savefig()
            show()
            close('all')
            
        elif self.save==False:  
            # Extract SL story
            t, e = readfile(
                dico.path_SLcurves+self.input_vars['SLstory__RSLin'])

            # Create the profile plot
            fig, ax = plt.subplots(figsize=(10,6))
            # Create the initial slope, based on output offset
            init_slope = (self.ds_out.grid__slopi.values * self.ds_out.x.values
                          + self.ds_out.profile__z[0].values)
            # Define bounds
            sup = int(self.ds_out.profile__xmax) + 1000
            inf = int(self.ds_out.profile__xmin) - 1000
                        
            if compare==False:
                # Plot init topo in dashed line
                ax.plot(init_slope[inf:sup], color='black',
                        linestyle='--', linewidth=0.5, label='initial topo')
                # Plot modelled topo
                ax.plot(self.ds_out.profile__z[inf:sup], color='black',
                        label='modelled topo')
                plt.xlim(0, sup-inf)
                
            elif type(compare)==str:
                # Extracts x and y obs in the topo files
                self.x_obs, self.y_obs = readfile(
                    os.path.abspath('../Topo_obs/'+compare))
                self.x_obs -= self.x_obs[0]
                self.align()
                ax.plot(self.x, self.y, label='modelled', color='black')
                ax.plot(self.x_obs, self.y_obs, label='observed')
                plt.legend()
                
            else:
                raise Exception("Wrong format for 'compare'. Must be string format for path")
            
            plt.xlabel('Distance (m)')
            plt.ylabel('Elevation (m)')
            plt.legend()
            
            # Add SL plot in subplot
            inset_ax = fig.add_axes([0.55, 0.18, 0.35, 0.25])
            inset_ax.plot(t, e, color='blue')
            inset_ax.set_title(self.input_vars['SLstory__RSLin'])
            inset_ax.set_xlim(0, max(t))
            inset_ax.set_xlabel('Time (ky)')
            inset_ax.set_ylabel('SL elevation (m)')   
            
        else:
            raise Exception("Wrong format for 'layers' variable. Should be True or False")

    def align(self):
        ipmin = min(self.x_obs)
        ipmax = max(self.x_obs)
        y_obs_min = min(self.y_obs)

        # Extract x and y from model output
        x = self.ds_out.profile__z.x.values
        y = self.ds_out.profile__z.values
        
        # Extract the index in y of the first element >= to the min value in y_obs.
        index_min = np.argmax(y >= y_obs_min) 
        # Use this index for the x starting value.
        x_start = x[index_min]
        
        ipmod = interp1d(x, y) # Interpolate y as a function of x.
        # Generate new continuous x values from x_start to max x value in topo_obs.
        # with the step defined in Inputs.inversion_params.
        self.x = np.arange(x_start, x_start - ipmin + ipmax, 1)
        self.y = ipmod(self.x) # Interpolation of the vertical values on new x_n axis.
        self.x = self.x - self.x[0] # Remove the offset to have x axis starting at 0.
        
REEF = REEF()

#### Parameterization

In the following cells, you only have to modify the parameters inside the function 'REEF.reef_params' or 'REEF.eros_params'. You will have the following parameters for the model, which are all in meter and years :
 - U : vertical movement (positive is uplift, negative is subsidence)
 - slop : the initial slope of the substrate (in m/m, put e-2 to have in %)
 - Gm : Reef growth rate 
 - Ev : eroded flux
   
If you want, you can also add :
 - hmax : maximum depth for reef growth, default to 20 m
 - Dbar : reef width growth factor, default to 100 m
 - hwb : wave base depth for erosion, default to 3 m
   
SL parameter can be set to a sea-level curve file name, stored in Library/RSLcurves/ here we will mostly use :
 - 'Holocene-SL.dat' : simple Holocene SL rise (linear)
 - 'Waelbroeck2002-137ky.dat' : sea-level curve to model MIS 5e and after mostly
 - 'Bintanja2005.dat' : 1 My of sea-level records, for the longest profiles

# Simple Holocene sea-level rise

For the following cells, 3 functions are called : 
 - reef_params, which defines the model parameters
 - run, to run the model
 - plot, to plot the output

For this part, you only have to run the cells. You can modify some values in reef_params if you want.

## Erosive terraces

Here the model is only erosive, without reef growth. 

In [ ]:
# Set the variables
REEF.eros_params(SL = 'Holocene-SL.dat',
                U = 0.6e-3,
                slop = 6e-2,
                Ev = 250e-3)

# Run the model
ds_out = REEF.run(dt = 200, save = False, construction=False)

# Plot the output
REEF.plot()

## Coral reef terraces

### No vertical movement

In [ ]:
# Set the variables
REEF.reef_params(SL = 'Holocene-SL.dat',
                U = 0e-3,
                slop = 6e-2,
                Gm = 9e-3,
                Ev = 60e-3)

# Run the model
REEF.run(dt = 200, save = True)

# Plot the output
REEF.plot()

### Subsidence

In [ ]:
# Set the variables
REEF.reef_params(SL = 'Holocene-SL.dat',
                U = -0.6e-3,
                slop = 6e-2,
                Gm = 9e-3,
                Ev = 60e-3)

# Run the model
REEF.run(dt = 200, save = True)

# Plot the output
REEF.plot()

### Uplift

In [ ]:
# Set the variables
REEF.reef_params(SL = 'Holocene-SL.dat',
                U = 0.6e-3,
                slop = 6e-2,
                Gm = 9e-3,
                Ev = 60e-3)

# Run the model
REEF.run(dt = 200, save = True)

# Plot the output
REEF.plot()

## Compare outputs

<img src="../Images/Profile-reef_no_mov.png" alt="title" width="350"> <img src="../Images/Profile-reef_subside.png" alt="title" width="350"> <img src="../Images/Profile-reef_uplift.png" alt="title" width="350">

# Reconstructing an existing geomorphology

## MIS 5e topographic profile

Here, the job is to reconstruct the following topographic profile :

<img src="../Images/Profile_Cape_MIS5.png" alt="title" width="550">

This coastline has been shaped since the MIS 5e (~120 ka), by wave erosion and coral growth. This profile can be reconstructed with our model. Try to reconstruct it by changing the parameters after considering the following questions :

 - From the profile, is it possible to approximate the eroded flux? The reef growth rate?
 - What is the local vertical movement (uplift/subsidence)?
 - Can you extract an approximation of the initial slope of the substrate from the profile? Altitudes from -130 m to 70 m
 - Can you give an approximation of the vertical movement? Hint : the highest topography (~60 m) is dated at 120 ka and the seal-level was similar than present day.



In [ ]:
# Set the variables
REEF.reef_params(SL = 'Waelbroeck2002-137ky.dat',
                U = e-3, 
                slop = e-2,
                Gm = e-3, # between 3 and 10 e-3
                Ev = e-3) # between 20 and 150 e-3

# Run the model
ds_out = REEF.run(dt = 200, save = False, construction=True)

# Plot the output
REEF.plot(compare='Topo_Cape_MIS5.dat')

## Erosive terraces
Here, the job is to reconstruct the following topographic profile :

<img src="../Images/Topo_Corinth1.png" alt="title" width="550">

This marine terraces sequence is located in the Gulf of Corinth, in Greece, and is only erosive.

!! Reconstructing the profile between 200 m and 340 m of altitude is enough !! Other altitudes are surely not reconstructible with our model, as it might be affected by a particular hydrodynamic setting.


### Some questions :

 - From the profile, is it possible to approximate the eroded flux? 
 - What is the local vertical movement (uplift/subsidence)?
 - Can you extract an approximation of the initial slope of the substrate from the profile?
 - Given the following sea-level curve, can you give an approximation of the vertical movement? Hint : a sample at 350 m altitude was dated at 400 ky and the sea-level was similar than present day.

In [ ]:
# Set the variables
REEF.eros_params(SL = 'Spratt2016-430k.dat',
                U = e-3, # Try between 0.85 and 0.92 mm/y
                slop = e-2, 
                Ev = e-3, # Between 100 and 300e-3
                hwb = 8)

# Run the model
ds_out = REEF.run(dt = 200, save = False, construction=False)

# Plot the output
REEF.plot(compare='TopoObs_Corinth_Prof1.dat')

## More complex profile

To have a better idea of the complexity of this system, you can try with the following profile :

<img src="../Images/Sumba_view.png" alt="title" width="400"> <img src="../Images/Topo_Cape.png" alt="title" width="400">

This is Sumba island, in Indonesia. A benchmark for long coral reef terraces records, as the oldest corals (500 m altitude) were dated at 1 Ma.

<img src="../Images/SL_Bintanja.png" alt="title" width="800">



#### From your deductions, test some values in the reef_params function.
 - U : vertical movement (positive is uplift, negative is subsidence)
 - slop : the initial slope of the substrate
 - Gm : Reef growth rate 
 - Ev : eroded flux
   
Everything in meter and year.

You can try with : <br>
Gm between 3e-3 and 10e-3 <br>
Ev between 50e-3 and 100e-3

In [ ]:
# Set the variables
REEF.reef_params(SL = 'Bintanja2005.dat',
                U = e-3,
                slop = e-2,
                Gm = e-3,
                Ev = e-3)

# Run the model
ds_out = REEF.run(dt = 400, save = False, construction=True)

# Plot the output
REEF.plot(compare='Laundi_dGPS_Denovan.dat')